In [134]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [135]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df_species = train_df.pop('species')
test_df_species = test_df.pop('species')


0      2
1      1
2      1
3      2
4      2
      ..
135    2
136    1
137    1
138    1
139    1
Name: species, Length: 140, dtype: int64

In [136]:
feature_columns = []
# for feature_name in train_df.keys():
for feature_name in ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

In [137]:
def make_input_fn(data, label, epoch=100, shuffle=True, batch_size=232):
  def inp():
    dataset = tf.data.Dataset.from_tensor_slices((dict(data), label))
    if shuffle:
      dataset = dataset.shuffle(1000)
    dataset = dataset.batch(batch_size).repeat(epoch)
    return dataset
  return inp

train_fn = make_input_fn(train_df, train_df_species)
test_fn = make_input_fn(test_df, test_df_species, epoch=1, shuffle=False)

In [143]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[30, 10],
    n_classes=3
)

classifier.train(
    train_fn,
    steps=5000
)
ac1 = classifier.evaluate(test_fn)
print("DNN accuracy:", ac1['accuracy'])

linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns, n_classes=3)
linear_est.train(train_fn)

ac2 = linear_est.evaluate(test_fn)
print("Linear accuracy:", ac2['accuracy'])

DNN accuracy: 0.3
Linear accuracy: 0.9


In [ ]:
classifier.evaluate(test_fn)

In [ ]:
def input_fn_no_lable(data, batch_size=256):
  def inpp():
    ds = tf.data.Dataset.from_tensor_slices((dict(data)))
    ds = ds.batch(batch_size)
    return ds
  return inpp

for _ in range(10):
  predict = {}
  for feature in train_df.keys():
    valid = True
    while valid:
      val = input(feature + ": ")
      valid =  val.isdigit()
    predict[feature] = [float(val)]


  get_predict_fn = input_fn_no_lable(predict)

  prediction1 = classifier.predict(get_predict_fn)
  prediction2 = linear_est.predict(get_predict_fn)
  l = list(prediction1)
  l2 = list(prediction2)
  cls = {
      '0': 'Setosa',
      '1': 'Versicolor',
      '2': 'Virginica'
  }
  print("Prediction 1(DNN):",cls[str(l[0]['class_ids'][0])], max(l[0]['probabilities']))
  print("Prediction 2(Lin):",cls[str(l2[0]['class_ids'][0])], max(l2[0]['probabilities']))
